In [1]:
import time
import numpy as np
import pandas as pd
import pyhodlrlib

In [14]:
# How to create a dataset for a regression problem
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=20000, n_features=10, noise=1, random_state=42)
N=len(X)


In [15]:
# View the some sample rows classification dataset
X=pd.DataFrame(X,
             columns=["x"+ str(i) for i in range(X.shape[1])])

In [16]:
y=X.apply(lambda row: row.x1/(row.x2*row.x3)-
                                  (row.x4*5)+3*(row.x5)+row.x7+3*(row.x6)+row.x8/row.x9+(row.x0), axis = 1)

In [17]:
X.shape

(20000, 10)

In [18]:
X=np.array(X)

In [19]:
y = np.expand_dims(y, axis=-1)
y.shape

(20000, 1)

In [20]:
def rbf_kernel(X1, X2):
    n_samples_X1, n_features = X1.shape
    n_samples_X2, _ = X2.shape
    kernel_matrix = np.zeros((n_samples_X1, n_samples_X2))

    for i in range(n_samples_X1):
        for j in range(n_samples_X2):
            squared_distance = np.linalg.norm(X1[i] - X2[j])
            kernel_matrix[i, j] = np.exp(-0.5* (squared_distance)**2)

    return kernel_matrix

In [21]:
type(X)

numpy.ndarray

In [22]:
import time
t0 = time.time()
#K2 = Kernel2(N2)
#A12 = K2.GP_hod(X1, y1, X2)
A11=rbf_kernel(X,X)
from scipy import linalg
x = linalg.solve(A11, y)
print(" time:", time.time()-t0)

 time: 5131.639305114746


In [23]:
# Returning the Gaussian Kernel:
class Kernel1(pyhodlrlib.HODLR_Matrix):
    def getMatrixEntry(self, i, j):
        dist=np.linalg.norm(X[i] - X[j])
        #print(dist)
        return np.exp(-0.5*(dist)**2)
         
    def GP_hod(self,X1, y, X2):
        # What we are doing here is explicitly generating 
        # the matrix from its entries
        A11=K1.getMatrix(0,0,N,N)
        #print(A11,"A11")
        return A11
t0 = time.time()
#print("A11",A12,A12.shape)
K1 = Kernel1(N)
A11 = K1.GP_hod(X, y, X)
eps = 1e-12
# If we are assembling a symmetric matrix:
is_sym = True
# If we know that the matrix is also PD:
# By setting the matrix to be symmetric-positive definite, 
# we trigger the fast symmetric factorization method to be used
# In all other cases the fast factorization method is used
is_pd = True
# Creating the HODLR object:
#print(type(A11))
M=N//5

T = pyhodlrlib.HODLR(N, M, eps)
T.assemble(K1, 'rookPivoting', is_sym, is_pd)
# Factorize elements of the tree:
T.factorize()
# Solving for x in A x = b:
x_hodlr = T.solve(y)
    
print(time.time()-t0)

15388.508623123169


In [12]:
x

array([[  -8.54573192],
       [  59.85110639],
       [  15.34594327],
       ...,
       [  -5.69235786],
       [ -74.60131908],
       [-103.54484362]])

In [13]:
x_hodlr

array([[  -8.54573192],
       [  59.85110639],
       [  15.34594327],
       ...,
       [  -5.69235786],
       [ -74.60131908],
       [-103.54484362]])

In [26]:

# import module
from tabulate import tabulate
 
# assign data
mydata = [
    [ 4853.41,19096.183 ], 
    [5131.63,15388.508 ]
    
]
 
# create header
head = ["np.linalg", "HODLR"]
 
# display table
print(tabulate(mydata, headers=head, tablefmt="grid"))

+-------------+---------+
|   np.linalg |   HODLR |
+=============+=========+
|     4853.41 | 19096.2 |
+-------------+---------+
|     5131.63 | 15388.5 |
+-------------+---------+
